# Section 3 - Identifying features that may drive outcomes

## Example 3.2
**Application 3.2**: What features might drive differential translation rates as measured by ribosome profiling when specific amino acids are in the A- and P-sites of the ribosome?

* Translation is a key biological process during which ribosomes synthesize proteins based on messenger RNA (mRNA) templates
* During translation elongation, amino acids are added one at a time to the nascent protein (**Figure 3.2.1**)
* The speed at which amino acids are added can be an important factor in determining if a protein will fold and function or misfold and malfunction

![](../images/Ribosome_mRNA_translation_en.svg.png)

**Figure 3.2.1** *The ribosome has three sites that accommodate tRNA: the A-, P- and E-sites. The ribosome ratchets along the mRNA, presenting different mRNA codons for decoding by aminoacyl-tRNA (aa-tRNA) at the A-site and catalyzing peptide bond formation between the nascent protein bound to the P-site tRNA and the amino acid bound to the A-site aa-tRNA. The E-site binds the deacylated tRNA before it exits the ribosome. Reproduced from https://en.wikipedia.org/wiki/Translation_(biology)*
  
* Many different factors are thought to influence the speed of translation
* In this application, we will explore using LASSO to determine which physicochemical properties are most useful in predicting how different combinations of amino acids and tRNA in the A- and P-sites of the ribosome influence translation speed


### Step 0 - Load libraries

In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, train_test_split, cross_validate
from sklearn.metrics import roc_auc_score, balanced_accuracy_score
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt

### Step 1 - Load the data & explore
* In this instance, the features and outcome are in two separate files

In [ ]:
# data7_features & data7_outcomes are both DataFrame objects
data_path      = "/home/jovyan/data-store/data/iplant/home/shared/NCEMS/BPS-training-2025/"
data7_features = pd.read_csv(data_path+"ta_transformed_features.csv")
data7_outcomes = pd.read_csv(data_path+"ta_transformed-data_targets.csv")

# both of these DataFrames have an unneeded first column, so we will remove it
data7_features = data7_features.iloc[:, 1:]
data7_outcomes = data7_outcomes.iloc[:, 1:]

print ("Summary of the features:\n")
display(data7_features.head(10))
display(data7_features.info())

print ("\nSummary of the outcomes:\n")
display(data7_outcomes.head(10))
display(data7_outcomes.info())

* What we are trying to predict is `Speed`, which is `0` if translation is faster for the given pair of amino acids in the A- and P-sites than average and `1` if it is slower than average
* These values were derived from ribosome profiling, a next-generation sequencing technique that specifically sequences fragments of mRNAs that are covered by the ribosome
* We will attempt to model `Speed` using the set of 629 features in `data7_features`
* Let's make sure that the features have each been scaled correctly

In [ ]:
print ("\nInformation about mean and standard deviation of parameters:")
data7_features.describe()

* We can see that the features have already been scaled - they have a mean of zero and standard deviation of one
* Finally, we need to check the balance of the outcome classes

In [ ]:
# calculate counts per outcome class
class_counts = data7_outcomes["Speed"].value_counts()
print (class_counts)

* We can see that the `0` class accounts for 54% of that data and the `1` class accounts for 46% of the data
* This is reasonably well balanced, but we should remain aware of the class imbalance all the same

### Step 2 - Prepare data for model building

* As in **Example 3.1**, we will use k-fold cross-validation with a grid search over *λ*

In [ ]:
# set random seed to achieve reproducible results
random_seed = 1

# number of folds for cross-validation
Nfolds      = 5

# define feature and outcome data sets; note that we need to drop some non-numerical columns from the feature space
X           = data7_features.drop(["asitetrna", "psitetrna", "asiteaa", "psiteaa"], axis=1)
y           = data7_outcomes["Speed"]

# reserve 20% of data for final testing after hyperparameter tuning
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.2, random_state=random_seed, stratify=y)

# set up k-fold cross-validation with outcome stratification
kf          = StratifiedKFold(n_splits=Nfolds, shuffle=True, random_state=random_seed)

# define a range of lambda values to be used in our grid search
lambda_vals = np.logspace(-1, 4, 6)

### Step 3 - Optimize λ
* We are now ready to run cross-validation for each value of *λ* and decide which value we want to use for our final model

In [ ]:
# record the start time
startTime    = datetime.now()

# maximum number of iterations to be run
max_iter     = 20000

# setup dictionary to store results for each value of lambda
results_dict = {}

# loop over lambda values
for lambda_val in lambda_vals:

    # setup logistic regression model
    model                    = LogisticRegression(penalty="l1", solver="saga", 
                                                  max_iter=max_iter, C=1/lambda_val)

    # run cross-validation for current lambda_val
    cv_results               = cross_validate(model, X_train, y_train, cv=kf, return_estimator=True, 
                                              scoring=['balanced_accuracy', 'roc_auc'], n_jobs=-1)

    # store results for later
    results_dict[lambda_val] = cv_results

    # calculation elapsed time and print it to the screen
    elapsed_sec              = (datetime.now() - startTime).total_seconds()
    print(f"{lambda_val:10.4f} {elapsed_sec:10.2f} s")

* Let's assess performance and number of features as a function of *λ*

In [ ]:
# sort the lambda values
lambda_vals = sorted(results_dict.keys())

# initialize lists to store the aggregated metric means and standard deviations
bal_acc_means, bal_acc_stds = [],[]
auroc_means, auroc_stds     = [],[]
nonzero_means, nonzero_stds = [],[]

# loop over each lambda and compute metrics
for lambda_val in lambda_vals:
    
    cv_results     = results_dict[lambda_val]
    
    # extract balanced accuracy and AUROC scores
    test_bal_acc   = cv_results['test_balanced_accuracy']
    test_roc_auc   = cv_results['test_roc_auc']
    
    # compute mean and standard deviation
    mean_bal_acc   = np.mean(test_bal_acc)
    std_bal_acc    = np.std(test_bal_acc, ddof=1)
    mean_roc_auc   = np.mean(test_roc_auc)
    std_roc_auc    = np.std(test_roc_auc, ddof=1)
    
    # compute number of non-zero coefficients for each fold
    nonzero_counts = [np.count_nonzero(estimator.coef_[0]) for estimator in cv_results['estimator']]
    mean_nonzero   = np.mean(nonzero_counts)
    std_nonzero    = np.std(nonzero_counts, ddof=1)
    
    # Append the computed metrics to the corresponding lists
    bal_acc_means.append(mean_bal_acc)
    bal_acc_stds.append(std_bal_acc)
    auroc_means.append(mean_roc_auc)
    auroc_stds.append(std_roc_auc)
    nonzero_means.append(mean_nonzero)
    nonzero_stds.append(std_nonzero)

# print summary information to screen
header = ("Lambda".ljust(12) + "Balanced Acc (mean ± std)".ljust(27) +
          "AUROC (mean ± std)".ljust(30) + "Non-zero Coeffs (mean ± std)")
print(header)

for i, lambda_val in enumerate(lambda_vals):
    nonzero_str = f"{nonzero_means[i]:10.1f} ± {nonzero_stds[i]:10.1f}"
    print(f"{lambda_val:10.4f}\t" f"{bal_acc_means[i]:0.3f} ± {bal_acc_stds[i]:0.3f}\t\t"
          f"{auroc_means[i]:0.3f} ± {auroc_stds[i]:0.3f}\t\t" f"{nonzero_str}")

# create summary plots
plot_color  = "#004488"
error_color = "#BB5566"
fig, axes   = plt.subplots(3, 1, figsize=(8, 6), sharex=True)

# plot Balanced Accuracy
axes[0].errorbar(lambda_vals, bal_acc_means, yerr=bal_acc_stds, fmt='o-', capsize=5, color=plot_color, ecolor=error_color)
axes[0].set_xscale('log')
axes[0].set_ylabel('Balanced Accuracy')
axes[0].set_ylim(0.45, 1.0)
axes[0].set_yticks([0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
axes[0].set_title('Balanced Accuracy vs Lambda')

# plot AUROC
axes[1].errorbar(lambda_vals, auroc_means, yerr=auroc_stds, fmt='o-', capsize=5, color=plot_color, ecolor=error_color)
axes[1].set_xscale('log')
axes[1].set_ylabel('AUROC')
axes[1].set_ylim(0.45, 1.0)
axes[1].set_yticks([0.5, 0.6, 0.7, 0.8, 0.9, 1.0])
axes[1].set_title('AUROC vs Lambda')

# plot number of non-zero coefficients
axes[2].errorbar(lambda_vals, nonzero_means, yerr=nonzero_stds, fmt='o-', capsize=5,color=plot_color, ecolor=error_color)
axes[2].set_xscale('log')
axes[2].set_ylabel('# of Non-Zero Coefficients')
axes[2].set_title('# of Non-Zero Coefficients vs Lambda')
axes[2].set_ylim(-25, 350)
axes[2].set_xlabel('Lambda')

# annotate each point with the mean number of non-zero coefficients (to one decimal)
for i, lambda_val in enumerate(lambda_vals):
    axes[2].annotate(f"{nonzero_means[i]:.1f}", (lambda_val, nonzero_means[i]), 
                     textcoords="offset points", xytext=(5, 5), fontsize=9, color='black')

plt.tight_layout()
plt.show()

* This plot and the associated data indicate that *λ* = 1 provides the best model performance based on balanced accuracy & AUROC
* As in **Exercise 3.1**, we will use a larger value of *λ* = 10 to limit the size of the non-zero feature space, improving interpretability

### Step 4 - Build & test the final model

In [ ]:
# choose our preferred value of lambda
final_lambda        = 10.

# setup the model
final_model         = LogisticRegression(penalty="l1", solver="saga", max_iter=max_iter, C=1/final_lambda)

# fit the model to the data
final_model.fit(X_train, y_train)

# evaluate the final model on the holdout dataset
y_holdout_pred_prob = final_model.predict_proba(X_holdout)[:, 1]
y_holdout_pred      = final_model.predict(X_holdout)
holdout_auroc       = roc_auc_score(y_holdout, y_holdout_pred_prob)
holdout_bal_acc     = balanced_accuracy_score(y_holdout, y_holdout_pred)

print ("Performance on holdout data\n")
print("Holdout AUROC             :", '%.3f' %holdout_auroc)
print("Holdout Balanced Accuracy :", '%.3f' %holdout_bal_acc, "\n")

# extract the nonzero coefficients
coef                = final_model.coef_.flatten()
nonzero_indices     = coef != 0
nonzero_coefs       = coef[nonzero_indices]
nonzero_features    = X_train.columns[nonzero_indices]

# sort coefficients by absolute magnitude in descending order
sorted_indices      = abs(nonzero_coefs).argsort()[::-1]
sorted_features     = nonzero_features[sorted_indices]
sorted_coefs        = nonzero_coefs[sorted_indices]

# print nonzero coefficients
print(len(sorted_coefs), "Nonzero Coefficients (sorted by magnitude)\n")
for feature, value in zip(sorted_features, sorted_coefs):
    print(feature.ljust(26) + ": " + "%.5f" % value)

### Step 5 - Assess the results
* When you are done, consider the following questions:
    * If you were tasked with using the results of this analysis to formulate a hypothesis about the features that are critical for determining translation speed, where would you start?
    * If we were satisfied with lower accuracy but wanted a smaller number of features to interpret, how could we achieve this?
* Use the QR code below to test your knowledge

![](../images/section-3-example-2.png)

[Quiz Link](https://forms.gle/KVoQcVgk9rqHitpV7)